Scripts to practise making ATom data comparable with UKCA data. Later on, this will be done properly with the SSP AMIP suites. Using my outputs from the test suite for now.

See also https://github.com/theabro/ukca/blob/master/Tutorials/UMvn13.0/notebooks/iris_toz01.ipynb

In [1]:
!pip install -q scitools-iris

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 91.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.1/518.1 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [89]:
import iris
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# This step takes several minutes. Better to load individual chunks than the whole thing. See stash codes text file.
p_file = '/content/drive/MyDrive/Documents/AI4ER/PhD/Photolysis_data/atmosa.pb1988sep'
#UKCA_data = iris.load(p_file)
UKCA_O3 = iris.load_cube(p_file,iris.AttributeConstraint(STASH='m01s50i567')) # O3 --> O(1D)

/usr/local/lib/python3.10/dist-packages/iris/fileformats/pp.py:1574: UserWarning: has_year_zero kwarg ignored for idealized calendars (always True)
  self._t1 = cftime.datetime(
/usr/local/lib/python3.10/dist-packages/iris/fileformats/pp.py:1609: UserWarning: has_year_zero kwarg ignored for idealized calendars (always True)
  self._t2 = cftime.datetime(
/usr/local/lib/python3.10/dist-packages/cf_units/__init__.py:402: UserWarning: has_year_zero kwarg ignored for idealized calendars (always True)
  dates = np.array([dt and dt.replace(microsecond=0) for dt in dates])
/usr/local/lib/python3.10/dist-packages/iris/fileformats/rules.py:353: UserWarning: Unable to create instance of HybridHeightFactory. The source data contains no field(s) for 'orography'.
  warnings.warn(msg.format(factory=factory_name))


In [4]:
'''
print(type(UKCA_data))
print(UKCA_data.shape)
print(UKCA_data)
'''
print(type(UKCA_O3)) # iris cube
print(UKCA_O3.shape) # 47 t, 85 z, 144 y, 192 x
print(UKCA_O3)

<class 'iris.cube.Cube'>
(47, 85, 144, 192)
m01s50i567 / (unknown)              (time: 47; model_level_number: 85; latitude: 144; longitude: 192)
    Dimension coordinates:
        time                             x                       -             -               -
        model_level_number               -                       x             -               -
        latitude                         -                       -             x               -
        longitude                        -                       -             -               x
    Auxiliary coordinates:
        forecast_period                  x                       -             -               -
        level_height                     -                       x             -               -
        sigma                            -                       x             -               -
    Scalar coordinates:
        forecast_reference_time     1988-09-01 00:00:00
    Attributes:
        STASH            

In [5]:
# The UKCA test suite simulates 1st and 2nd September 1988.
# The most similar time of year in the ATom data is 23rd August 2016.

# Open the .csv of all the ATom data which I have already pre-processed in the script, ATom_J_data.
ATom_file = '/content/drive/MyDrive/Documents/AI4ER/PhD/Photolysis_data/ATom_MER10_Dataset.20210613/photolysis_data.csv'
ATom_data = pd.read_csv(ATom_file, index_col=0) # shape = 138454, 49. 138454 time steps, 49 = chemicals + spatial dimensions.

In [6]:
# Pick out just the 22nd and 23rd August indices separately.
A_day1 = ATom_data[ATom_data.index.str.contains('2016-08-22')] # shape = 1994, 49. 1994 time steps for 22/8/16, 49 = chemicals + spatial dimensions.
A_day1_O3 = A_day1[['G_LAT', 'G_LONG', 'G_ALT', 'jO3_O2_O1D_CAFS']] # shape = 1994, 4.

A_day2 = ATom_data[ATom_data.index.str.contains('2016-08-23')] # shape = 1765, 49. 1765 time steps for 23/8/16, 49 = chemicals + spatial dimensions.
A_day2_O3 = A_day2[['G_LAT', 'G_LONG', 'G_ALT', 'jO3_O2_O1D_CAFS']] # shape = 1765, 4.

# Check what the time of day range is for both.
print(A_day1_O3.index[0]) # 12:53:40
print(A_day1_O3.index[-1], '\n') # 18:51:40
print(A_day2_O3.index[0]) # 15:05:40
print(A_day2_O3.index[-1]) # 20:00:20

# Trim so that all arrays are over the same times. # 15:05:40 to 18:51:40
A_day1_O3 = A_day1_O3.loc['2016-08-22 15:05:40':'2016-08-22 18:51:40'] # shape = 1205, 4. type = pandas dataframe.
A_day2_O3 = A_day2_O3.loc['2016-08-23 15:05:40':'2016-08-23 18:51:40'] # shape = 1353, 4. type = pandas dataframe.

2016-08-22 12:53:40
2016-08-22 18:51:40 

2016-08-23 15:05:40
2016-08-23 20:00:20


In [94]:
# shape = 47 t, 85 z, 144 y, 192 x. type = iris cube.

# Check what time zones ATom and UM are using. ATom: UTC. UM: UTC.
# Get the UM data for the same time of day for both 1st and 2nd sep separately.
U_day1_O3 = UKCA_O3[13:18]
U_day2_O3 = UKCA_O3[37:42] # shape = 5 t, 85 z, 144 y, 192 x. type = iris cube.

times = ['2016-08-22 15:05:40', '2016-08-22 16:00:00', '2016-08-22 17:00:00', '2016-08-22 18:00:00', '2016-08-22 18:51:40']
U_day, A_day = [], []

for iTime in range(len(times)):

  t = U_day1_O3[iTime]

  # Check what units of altitude they're using. ATom: m. UM: m.
  # Find out what the altitudes are for ATom and select the right altitudes of UKCA data.
  Talt = A_day1_O3.loc[times[iTime]]['G_ALT']
  Ualts = t.coord('level_height').points
  diffs = np.absolute(Ualts-Talt)
  iAlt = diffs.argmin()
  t = U_day1_O3[iTime,iAlt]

  # Check what units of latitude they're using. ATom: degrees +N. UM: degrees.
  # Find out what the latitudes are for ATom and select the right latitudes of UKCA data.
  Tlat = A_day1_O3.loc[times[iTime]]['G_LAT']
  Ulats = t.coord('latitude').points
  diffs = np.absolute(Ulats-Tlat)
  iLat = diffs.argmin()
  t = U_day1_O3[iTime,iAlt,iLat]

  # Check what units of longitude they're using. ATom: degrees +E in half circles (-180 to 180). UM: degrees in a full circle (0 to 360).
  # Find out what the longtitudes are for ATom and select the right longitudes of UKCA data.
  Tlong = A_day1_O3.loc[times[iTime]]['G_LONG']
  Ulongs = t.coord('longitude').points
  if Tlong < 0:
    Tlong = Tlong + 360
  diffs = np.absolute(Ulongs-Tlong)
  iLong = diffs.argmin()
  t = U_day1_O3[iTime,iAlt,iLat,iLong]

  U_day.append(t)
  A_day.append(A_day1_O3.loc[times[iTime]])

print(U_day[0])
print(A_day[0])

# Used Monsoon to check if sigma is the J rate. sigma is not J rate.
# compare the 2 ATom days with each other for O3.
# compare the 2 UM days with each other for O3.
# compare the ATom days with the UM days.
# Now put all the above into functions to work with different chemicals and times etc and try with NO2.

m01s50i567 / (unknown)              (scalar cube)
    Scalar coordinates:
        forecast_period             15.0 hours
        forecast_reference_time     1988-09-01 00:00:00
        latitude                    59.375 degrees
        level_height                2659.99935 m, bound=(2529.9995000000004, 2796.6666) m
        longitude                   285.9375 degrees
        model_level_number          19
        sigma                       0.7220069222008947, bound=(0.734543106256622, 0.7089441261204913)
        time                        1988-09-01 15:00:00
    Attributes:
        STASH                       m01s50i567
        source                      'Data from Met Office Unified Model'
        um_version                  '13.0'
G_LAT                58.951297
G_LONG              -74.854346
G_ALT              2609.580000
jO3_O2_O1D_CAFS       0.000028
Name: 2016-08-22 15:05:40, dtype: float64
